In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [3]:
# Data Augumentation

batch_size=16

training_datagenerator = ImageDataGenerator(rescale=1./255, horizontal_flip=True, 
        vertical_flip = True, shear_range=0.2, zoom_range=0.2, 
        width_shift_range=0.2, height_shift_range=0.2, validation_split=0.1)

In [4]:
#dividng data into training and validation

train = training_datagenerator.flow_from_directory('Training_Data', target_size=(224,224), color_mode='rgb',
                            class_mode= 'binary', batch_size=batch_size, subset='training')

validation = training_datagenerator.flow_from_directory('Training_Data', target_size=(224,224), color_mode='rgb',
                            class_mode= 'binary', batch_size=batch_size, subset='validation')

Found 12360 images belonging to 2 classes.
Found 1373 images belonging to 2 classes.


In [5]:
#CNN Architecture

#Initializing CNN
cnn = tf.keras.models.Sequential()

#Adding first layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

#Adding second layer
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

#Adding third layer
cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

#Flattening 
cnn.add(tf.keras.layers.Flatten())

#Fully connected layer
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Output layers
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [6]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       147712    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0

In [7]:
#compile and train

checkpoint = tf.keras.callbacks.ModelCheckpoint('Fire_and_Smoke.h5',
                                                monitor='val_loss', mode='min',
                                               save_best_only=True)
callbacks=checkpoint

In [ ]:
cnn.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit_generator(train, validation_data=validation, epochs=1,
                 steps_per_epoch=train.samples//batch_size,
                 validation_steps=validation.samples//batch_size,
                 callbacks=callbacks)

Instructions for updating:
Please use Model.fit, which supports generators.
 58/772 [=>............................] - ETA: 21:47 - loss: 0.2389 - accuracy: 0.9569

In [ ]:
#testing

from tensorflow.keras.models import load_model
cnn=load_model('Fire_and_Smoke.h5')

In [ ]:
cnn.summary()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import PIL
import cv2

In [ ]:
#cap = cv2.VideoCapture(0) 

#while cap.isOpened(): 
 #   ret,frame = cap.read() 
  #  if ret == True:
        frame = r 'Training_Data\Fire\14.jpg'
        test_image=image.load_img(frame, target_size=(224,224))
        test_image=image.img_to_array(test_image)
        test_image=test_image/255
        test_image=np.expand_dims(test_image, axis=0)
        result = cnn.predict_classes(test_image)

        Categories = ['Fire', 'Smoke']

        image_show=PIL.Image.open(frame)
        plt.imshow(image_show)

        plt.title(Categories[int(result[0][0])])
        plt.show()